### OPEN API 

부산광역시 버스정보안내기 현황 https://www.data.go.kr/data/15034014/openapi.do

In [25]:
import json
import ssl

from urllib.request import urlopen, Request
from urllib.parse import quote                  # kr lan urlencode func

ssl._create_default_https_context = ssl._create_unverified_context      # ssl 추가로 오류 제거 (colab)

def getRequestUrl(url):
    req = Request(url)

    try:
        res = urlopen(req)
        if res.getcode() == 200:
            return res.read().decode('utf-8')
    except Exception as e:
        print(e)
        return None

# stationName 정류소 이름
def getDataPortalSearch(stationName, type):
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    api_url += '?serviceKey=wGokgRxD1t3z5G4u7MsWumpoCeiWO8JM6yZ87rX1ELTO9nMSUuMOQjHj70rAzuopgyB1iLdKX0S9WK0RLs88bQ%3D%3D'
    api_url += '&pageNo=1'
    api_url += '&numOfRows=10'
    api_url += f'&stationLoc={stationName}'
    api_url += f'&resultType={type}'

    response = getRequestUrl(api_url)

    if response != None:
        return json.loads(response)

    return None


getDataPortalSearch(quote('롯데'), 'json')

<urlopen error [SSL: SSLV3_ALERT_ILLEGAL_PARAMETER] sslv3 alert illegal parameter (_ssl.c:992)>


In [44]:
import ssl
import json
import requests

from urllib.parse import quote, unquote, urlencode                  # kr lan urlencode func

def getRequestUrl(url):
    req = Request(url)

    try:
        res = urlopen(req)
        if res.getcode() == 200:
            return res.read().decode('utf-8')
    except Exception as e:
        print(e)
        return None

# stationName 정류소 이름
def getDataPortalSearch(stationName, type):
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    queryString = '?' + urlencode(
        {
            'serviceKey' : 'wGokgRxD1t3z5G4u7MsWumpoCeiWO8JM6yZ87rX1ELTO9nMSUuMOQjHj70rAzuopgyB1iLdKX0S9WK0RLs88bQ==',
            'pageNo' : '1',
            'numOfRows' : '10',
            'stationLoc' : stationName,
            'resultType' : type
        }
    )

    total_url = api_url + queryString
    response = requests.get(total_url, verify = False)           # verify = False 로 검증 끄기
    
    return response.text
        
try:
    result = getDataPortalSearch(input(), 'json')

    json_data = json.loads(result)                                   # 파일이 아니므로 loads
    station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item']

    import folium

    lat = []
    lng = []

    for item in station_data:
        lat.append(float(item['lat']))
        lng.append(float(item['lng']))

    scale = max(max(lat) - min(lat), max(lng) - min(lng))
    value = 18

    if scale >= 1 and scale <=1.5:
        value = 9
    elif scale >= 0.6 and scale <= 0.9:
        value = 10
    elif scale >= 0.5 and scale <= 0.1:
        value = 11
    else: value = 12

    print(value)

    lat = 0
    lng = 0

    for item in station_data:
        lat += float(item['lat'])
        lng += float(item['lng'])

    m = folium.Map(location = [lat / len(station_data), lng / len(station_data)], zoom_start = value)

    for item in station_data:
        stop_str = '<h4>' + item['stationLoc'] + '</h4> (' + item['stationNum'] + ') ' + item['addr']
        iframe = folium.IFrame(stop_str)
        popup = folium.Popup(iframe, min_width = 200, max_width = 200)
        folium.Marker(location = [item['lat'], item['lng']],
                    popup = popup).add_to(m)

except Exception as e:
    print('No data')

c:\DEV\Langs\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [45]:
m.add_child(folium.LatLngPopup())
m